
### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2020 Semester 1

## Assignment 1: Naive Bayes Classifiers

###### Submission deadline: 7 pm, Monday 20 Apr 2020

**Student Name(s):**    Shuyang Fan, Yiran Wang 

**Student ID(s):**     988301, 987751


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [263]:
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy.stats import mode 
from collections import Counter, defaultdict

In [264]:
data_set ="./datasets/bank.data"

# Read data from csv
def read_data(fileName):
    data = pd.read_csv(fileName, header=None)
    # print(data.head(5))
    return data

In [265]:
def handle_missing_value(data):
    # Make a copy of raw data
    copy = data.copy()
    # Drop rows contains question mark
    copy = copy[(copy.astype(str) != '?').all(axis=1)]
    # Extract label from data
    label = copy.iloc[:,-1]
    # Filling missing value with category mode for each column
    for i in range(copy.shape[1] - 1):
        copy.iloc[:,:i] = copy.iloc[:,:i].groupby(label).transform(lambda x: x.fillna(x.mode())) 
    # Print how many missing value have been handled 
    # print(data.isna().sum() - copy.isna().sum())
    # return the modified copy
    return copy

def binning(data):
    copy = data.copy()
    
    numeric_column = []
    for column in range(copy.shape[1] - 1):
        if (is_numeric_dtype(copy.iloc[:,column])):
            numeric_column.append(column)
        
        
    discretizer(copy, numeric_column, [3 for i in range(len(numeric_column))])
    #print(copy.head())
    return copy
    
def discretizer(X, column_index, bin_size):
    for index, column in enumerate(column_index):
        X.iloc[:,column] = pd.cut(X.iloc[:,column], bin_size[index])

def train_test_split(X, y, test_size=0.2):
    X_total = X.shape[0]
    assert(X_total == y.size)
    arr_rand = np.random.rand(X.shape[0])
    split = arr_rand < np.percentile(arr_rand, test_size*100)

    
    X_train = X[split]
    y_train = y[split]
    X_test =  X[~split]
    y_test = y[~split]
    return X_train, X_test, y_train, y_test

# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing
def preprocess(data):
    data = handle_missing_value(data)
    data = binning(data)
    
    # print(data)
    X = data.iloc[:,:-1]
    y = data.iloc[:,-1]

    return X, y


In [266]:
#Bayes calculate the product of prior and conditinoals,, take the max
#then predict, throw the X-test into the model
#then check the accuracy

In [267]:
def fine_key_with_max_value(dic):
    max_value = 0
    max_key = None
    for key in dic:
        if dic[key] > max_value:
            max_key = key
            max_value = dic[key]
    return max_key
fine_key_with_max_value({"a":1, "b":2})

'b'

In [272]:
class BayesClassifier():

    def train(self, X_train, y_train):
        self.X_train = X_train.copy()
        self.y_train = y_train.copy()
        self.possible_labels = np.unique(self.y_train)
        self.prior = self.get_prior(self.y_train)
        self.conditional_prob = self.compute_conditional_probability(self.X_train, self.y_train)
    
    def get_prior(self, y_train):
        train_inputs = y_train
        #counts is a list that stores number of each label accordingly
        labels, counts = np.unique(train_inputs, return_counts=True)
        prior = {}
        for i, label in enumerate(labels):
            prior[label] = float(counts[i])/len(train_inputs)
        return prior      

    def compute_conditional_probability(self, X_train, y_train):
        conditional_prob = defaultdict(lambda: defaultdict(dict))
        # Sepeate training instances by label
        grouped = X_train.groupby(y_train)

        for label in self.possible_labels:
            separated = grouped.get_group(label)
            for column_index in range(X_train.shape[1]):
                # Extract one attribute from group
                attribute = separated.iloc[:,column_index]
                # Find all possible values of this attribute
                possible_values = np.unique(X_train.iloc[:,column_index])
                # Call Counter to count the frequency of each value
                counts = Counter(attribute)
                for value in possible_values:
                    if value in counts:
                        conditional_prob[column_index][str(value)][label] = self.laplace_smoothing(counts[value],attribute.shape[0],np.size(possible_values))
                    else:
                        conditional_prob[column_index][str(value)][label] = self.laplace_smoothing(0, attribute.shape[0], np.size(possible_values))
        return conditional_prob
    
    def laplace_smoothing(self, freq, total, possible_value_count):
        return freq+1 / total +possible_value_count
    
    def predict(self, X_test):
        X_test_copy = X_test.copy()
        conditional_prob = self.conditional_prob
        priors = self.prior
        possible_labels = self.possible_labels
        predicted_outputs = []
    
        row, column = X_test_copy.shape
        for row_index in range(row):
            probability = defaultdict(float)
            
            for label in possible_labels:
                probability[label] = np.log(priors[label])
                
            for column_index in range(column):
                # Get the value of this attribute
                value = X_test_copy.iloc[row_index, column_index]
                # Get conditional probability
                for label in possible_labels:
                    probability[label] += np.log(conditional_prob[column_index][str(value)][label])
            # The prediced outcome is the lebel with the highest probability
            predicted_outputs.append(fine_key_with_max_value(probability))
        return predicted_outputs

In [273]:
data = read_data(data_set)
X,y = preprocess(data)

X_train, X_test, y_train, y_test = train_test_split(X,y)
train(X_train, y_train)
bayes = BayesClassifier()
bayes.train(X_train, y_train)
result = bayes.predict(X_train)
print(accuracy(np.array(result), y_train))

0.882879893828799


In [274]:
def random_baseline(X_train, y_train):
    labels = np.unique(y_train)
    y_baseline = [np.random.choice(labels) for i in range(len(y_train))]
    return y_baseline

In [275]:
def random_baseline(X_train, y_train):
    labels = np.unique(y_train)
    y_baseline = [np.random.choice(labels) for i in range(len(y_train))]
    return y_baseline

In [276]:
def zero_r_baseline(X_train, y_train):
    label = mode(y_train)
    return np.repeat(label, len(y_train))

In [277]:
def accuracy(y_predicted, y_truth):
    assert(y_predicted.size==y_truth.size)
    return np.sum(y_predicted == y_truth)/y_predicted.size

print(accuracy(np.array([1,2,3,4]), np.array([1,2,3,5])))

0.75


In [278]:
def label_confusion_matrix(y_predicted, y_truth, label):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for index in range(y_predicted.size):
        result =  y_predicted[index]
        if (result==label):
            if ((y_truth[index]) == label):
                TP +=1
            else:
                FP +=1
        else:
            if ((y_truth[index]) == label):
                FN += 1
            else:
                TN += 1
    return TP, TN, FP, FN
                
  
(TP, TN, FP, FN) = label_confusion_matrix(np.array([1, 0, 0, 1, 2]), np.array([1, 0, 1, 0, 2]), 2)  
print(TP, TN, FP, FN)    
    

1 4 0 0


In [ ]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels
def evaluate():
    return 

## Questions 


If you are in a group of 1, you will respond to question (1), and **one** other of your choosing (two responses in total).

If you are in a group of 2, you will respond to question (1) and question (2), and **two** others of your choosing (four responses in total). 

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions in respond to the question, but your formal answer should be added to a separate file.

### Q1
Try discretising the numeric attributes in these datasets and treating them as discrete variables in the na¨ıve Bayes classifier. You can use a discretisation method of your choice and group the numeric values into any number of levels (but around 3 to 5 levels would probably be a good starting point). Does discretizing the variables improve classification performance, compared to the Gaussian na¨ıve Bayes approach? Why or why not?

### Q2
Implement a baseline model (e.g., random or 0R) and compare the performance of the na¨ıve Bayes classifier to this baseline on multiple datasets. Discuss why the baseline performance varies across datasets, and to what extent the na¨ıve Bayes classifier improves on the baseline performance.

### Q3
Since it’s difficult to model the probabilities of ordinal data, ordinal attributes are often treated as either nominal variables or numeric variables. Compare these strategies on the ordinal datasets provided. Deterimine which approach gives higher classification accuracy and discuss why.

### Q4
Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy (you should implement this yourself and do not simply call existing implementations from `scikit-learn`). How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)

### Q5
Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the na¨ıve Bayes classifier? Explain why, or why not.

### Q6
The Gaussian na¨ıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in these datasets? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the NB classifier’s predictions.